In [3]:
from requests import get
from requests_html import HTMLSession
import scrapy
from scrapy.crawler import CrawlerProcess
import pandas as pd
import sqlite3
from pprint import pprint as print

In [11]:
def data_json(base_size,maximum_size,next_page=0):
    session=HTMLSession()
    url=f'https://api.divar.ir/v8/map-discovery/bbox/posts?lon1=50.780457955008785&lat1=35.43862123757947&lon2=51.702507060611595&lat2=35.94376848564657&nextPageId={next_page}&filters=building-age=0-10&filters=category=apartment-sell&filters=size={base_size}-{maximum_size}'
    response=session.get(url)
    print(response.json())
apartment=data_json(60,110)
apartment

{'next_page_id': '1689790031612300',
 'posts': [{'image_count': 3,
            'image_url': 'https://s100.divarcdn.com/static/thumbnails/AZ4uyoEW.webp',
            'lat': 35.741913,
            'lon': 51.285057,
            'preview_tile': '۷۲۵ میلیون',
            'subtitle1': '۷۲۵٬۰۰۰٬۰۰۰ تومان',
            'subtitle2': 'قیمت هر متر: ۷٬۹۶۷٬۰۰۰ تومان',
            'subtitle3': 'دقایقی پیش، آژانس گروه مشاورین املاک ققنوس',
            'title': '91 متر/ 5ساله/ سرمایه گذاری (شهرزیبا)',
            'token': 'AZ4uyoEW'},
           {'image_count': 6,
            'image_url': 'https://s100.divarcdn.com/static/thumbnails/AZ4yCDiw.webp',
            'lat': 35.82999,
            'lon': 50.99608,
            'preview_tile': '۶٫۲۰۰ میلیارد',
            'subtitle1': '۶٬۲۰۰٬۰۰۰٬۰۰۰ تومان',
            'subtitle2': 'قیمت هر متر: ۵۶٬۳۶۳٬۰۰۰ تومان',
            'subtitle3': 'دقایقی پیش، آژانس هلدینگ بزرگ مسکن آران',
            'title': 'واحد110متری /نوساز /فول امکانات/',
            'token': 'AZ4